In [ ]:
# regular experession support
import re
# platform independent file system path handling
from pathlib import Path
# pandas
import pandas as pd

# string identifier to connect chunks to a biopsy [jrwXXX] and section scan [s] #
sample = "jrw006_s1"
# path to directory of csv files output by point cloud generating script #
img_output = Path("/mnt/d/tda_workflow/_post_pointcloud_demo/alpha_raw+SECT__output/topology_results")
# list of all paths+names of csv files in this directory #
file_list = list(img_output.glob("*.csv"))

##
metadata_log = {}  # Dictionary to store metadata
master_df = pd.DataFrame() # Dataframe to store raw_ec data

for file_path in file_list:
    df = pd.read_csv(file_path)
    file_stem = file_path.stem
    # pull chunk boundaries from file stem
    z_coord = re.search(r"Z(\d+-\d+)", file_stem)
    y_coord = re.search(r"Y(\d+-\d+)", file_stem)
    x_coord = re.search(r"X(\d+-\d+)", file_stem)

    if z_coord and y_coord and x_coord:
        z_range = z_coord.group(1)
        y_range = y_coord.group(1)
        x_range = x_coord.group(1)
        chunk_id = f"Z{z_range}_Y{y_range}_X{x_range}"

        # append chunk info to metadataw dictionary
        metadata_log[chunk_id] = {
            "sample": sample,
            "z_range": z_range,
            "y_range": y_range,
            "x_range": x_range,
            "file_path": str(file_path), #store file path as string.
        }

        ### extract the column of interest (raw ec in this case) for the master df for FDA ###
        master_df[chunk_id] = df["Raw EC (3D)"]

    else:
        print(f"Skipping {file_path.name} due to missing Z, Y, or X range.")

master_df['Filtration Value'] = df['Filtration Value'] #add filtration value column.
print(master_df.head())
print(metadata_log)